# Model Ensemble
In this notebook, we will do the predictions using all our models and averaging the results.\
We will do that using two different approaches:
1. Averaging the softmax probabilities of the models and then taking the class with the highest probability.
2. Deciding the class based on the majority vote of the models.



In [1]:
import torch
import model as m
from data_loading import get_val_transforms, WeatherDataModule

In [2]:
model_keys = ["efficientnetb0", "efficientnetb1", "mobilenet", "resnet50", "swin"]
model_names = ["EfficientnetB0", "EfficientnetB1", "MobileNetV2", "ResNet50", "Swin-T"]
ckpt_paths = ["models/efficientnetb0-val_acc=0.93.ckpt", "models/efficientnetb1-val_acc=0.92.ckpt", "models/mobilenet-val_acc=0.90.ckpt", "models/resnet50-val_acc=0.93.ckpt", "models/swin-val_acc=0.93.ckpt"]

In [3]:
models = []
for model_name, model_key, ckpt_path in zip(model_names, model_keys, ckpt_paths):
    model = m.get_base_model(model_key, 11)
    state_dict = torch.load(ckpt_path, weights_only=True, map_location=torch.device('cpu'))['state_dict']
    for key in list(state_dict.keys()):
        state_dict[key.replace("model.", "")] = state_dict.pop(key)
    model.load_state_dict(state_dict,strict=False)
    models.append(model)

/home/joscha/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [4]:
datamodule = WeatherDataModule("./data/weather-dataset", 32, 1, get_val_transforms(), get_val_transforms())
datamodule.setup()
test_dataloader = datamodule.test_dataloader()
X_val = []
y_val = []
for images, labels in test_dataloader:
    X_val.append(images)
    y_val.append(labels)
X_val = torch.cat(X_val)
y_val = torch.cat(y_val)

In [5]:
predictions = []
X_val = X_val.to("cuda")
for model in models:
    print(f"Running model {model_names[models.index(model)]}")
    model.to("cuda")
    model.eval()
    model_predictions = []
    with torch.no_grad():
        for batch in X_val.split(32):
            model_predictions.append(model(batch))
        model_predictions = torch.cat(model_predictions)
        predictions.append(model_predictions)

Running model EfficientnetB0
Running model EfficientnetB1
Running model MobileNetV2
Running model ResNet50
Running model Swin-T


In [6]:
predictions = torch.stack(predictions)
final_predictions = torch.mode(predictions, dim=0).values
final_predictions = torch.argmax(final_predictions, dim=1)
y_val = y_val.to("cuda")
accuracy = (final_predictions == y_val).float().mean().item() * 100
print(f"Ensemble accuracy max vote: {accuracy:.1f}%")
predictions = predictions.mean(dim=0)
final_predictions = torch.argmax(predictions, dim=1)
accuracy = (final_predictions == y_val).float().mean().item() * 100
print(f"Ensemble accuracy mean vote: {accuracy:.1f}%")

Ensemble accuracy max vote: 93.6%
Ensemble accuracy mean vote: 93.9%
